In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

from models.vgg import VGG
from gsp_methods import *

import math
import sys 
sys.path.append('/data/users2/rohib/github/testing')
import utils_gsp.sps_tools as sps_tools
import utils_gsp.padded_gsp as gsp_pad

from torch.utils.tensorboard import SummaryWriter

In [3]:
class Args:
    lr = 0.1
    resume = False
    gsp_sps = 0.8
    gsp_int = 3
    start_epoch = -1

args = Args

writer = SummaryWriter()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Model
print('==> Building model..')
# net = VGG(depth=16, dataset='cifar10', batchnorm=True)
model = VGG('VGG19')
model = model.to(device)
model.gsp_training_mode = False
# model

==> Building model..


In [6]:
def get_model_layers(model):
    names = list()
    weight_l = list()
    shape_l = list()
    weight_d = dict()
    layers = dict()
    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            weight_d[name] = layer.weight
            weight_l.append(layer.weight)
            shape_l.append(layer.weight.shape)
            layers[name] = layer
            names.append(name)
            
    return names, weight_l, shape_l, weight_d, layers

best_acc
def gsp_sparse_training(model, args):
    # Additional Class Variables for GSP
    print(f"ARGS GSP INT: {args.gsp_int}")
    model.sps = args.gsp_sps
    model.curr_iter = 0
    model.start_gsp_epoch = -1
    model.gsp_int = args.gsp_int
    model.logger = None
    model.gsp_training_mode = True

    if args.resume:
        model.curr_epoch = args.start_epoch
        print(f"Current Epoch: {args.start_epoch}")
    else:
        model.curr_epoch = 0

gsp_sparse_training(model, args)

ARGS GSP INT: 3


In [7]:
names, weight_l, shape_l, weight_d, layers = get_model_layers(model)

In [8]:
bind_new_gsp_methods_to_model(model, args)

Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding the NEW forward layers!
Binding 

In [9]:
chk_name = 'gsp_block'
args.resume = False
if args.resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load(f'./checkpoint/{chk_name}.pth')
    model.load_state_dict(checkpoint['model'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [ ]:
model.gsp_training_mode = True
model.start_gsp_epoch = -1

print(f"gsp start epoch: {model.start_gsp_epoch}")
print(f"epoch: {model.curr_epoch} | iter: {model.curr_iter} | gsp_int: {model.gsp_int}")

images, target = next(iter(trainloader))

images = images.cuda('cuda:0', non_blocking=True)
target = target.cuda('cuda:0', non_blocking=True)

optimizer.zero_grad()
output = model(images)
loss = criterion(output, target)

model.curr_iter += 1
# get_abs_sps(model)

In [ ]:
loss.backward()
optimizer.step()

In [ ]:
sps_tools.sparsity(layers[names[9]].gsp_gate.flatten())

In [ ]:
gate_d = model.gate_d
matrix, ni_list = gsp_pad.pad_input_dict(gate_d)
len(gate_d)

In [ ]:
# gate_d[15][0:ni_list[15]]

In [ ]:
cnt = 0
for name, layer in model.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        trimmed_gate = gate_d[cnt][0:ni_list[cnt]]
        layer.gsp_gate.data = trimmed_gate
        # print(trimmed_gate.shape)
        cnt += 1

In [ ]:
layer.gsp_gate

In [ ]:
print(sps_tools.padded_sparsity(matrix, ni_list))

In [ ]:
xp_mat, ni_list = gsp_pad.groupedsparseproj(gate_d, 0.85, precision=1e-6, linrat=0.9)

In [ ]:
gsp_gate

In [ ]:
sps_tools.padded_sparsity(xp_mat, ni_list)

------------------
## Flow Through Network

In [ ]:
images, target = next(iter(trainloader))

images = images.cuda('cuda:0', non_blocking=True)
target = target.cuda('cuda:0', non_blocking=True)
print(images.shape)

In [ ]:
# model.features(images)
out1 = model.features[0](images)
print(f"images: {images.shape} || layer1: {model.features[0].weight.shape } || out: {out1.shape}")

out2 = model.features[3](out1)
print(f"inputs: {out1.shape} || layer2: {model.features[3].weight.shape} || out: {out2.shape}")

out3 = model.features[7](out2)
print(f"inputs: {out2.shape} || layer3: {model.features[7].weight.shape} || out: {out3.shape}")

out4 = model.features[10](out3)
print(f"inputs: {out3.shape} || layer4: {model.features[10].weight.shape} || out: {out4.shape}")

out5 = model.features[14](out4)
print(f"inputs: {out4.shape} || layer5: {model.features[14].weight.shape} || out: {out5.shape}")

In [ ]:
model.features[14].weight.shape

In [ ]:
# images, target = next(iter(trainloader))

# images = images.cuda('cuda:0', non_blocking=True)
# target = target.cuda('cuda:0', non_blocking=True)
# print(images.shape)

# model(images)

In [ ]:
a = torch.ones(2,2,4,2)
print(a)
b = torch.ones(2,2,2,1) * 2

In [ ]:
print(a @ b)

-------------
## Setup Training and related functions

In [10]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print( f"[{batch_idx}/{len(trainloader)}], Loss: {(train_loss/(batch_idx+1))} | Acc: {100.*correct/total} " \
                   f"grad norm-7: {torch.norm(model.features[7].gsp_gate.grad):.3f} | w_norm: {torch.norm(model.features[7].gsp_gate):.3f}")

In [11]:
def test(model, criterion, testloader, epoch, args):
    global best_acc
    model.eval()
    model.gsp_training_mode = False

    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    print(f"Accuracy: {acc}")

test(model, criterion, testloader, 1, args)

Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Conv2D forward
Modified NEW Linear forward


/home/users/rohib/anaconda3/envs/imagenet/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Accuracy: 10.0


In [12]:
train(2)


Epoch: 2
[0/391], Loss: 2.3510491847991943 | Acc: 7.8125 grad norm-7: 2.944 | w_norm: 1.363
[100/391], Loss: 2.3620583515356084 | Acc: 11.997215346534654 grad norm-7: 0.001 | w_norm: 3.145
[200/391], Loss: 2.207957200743073 | Acc: 15.127487562189055 grad norm-7: 0.000 | w_norm: 2.991
[300/391], Loss: 2.1229694234176333 | Acc: 16.805959302325583 grad norm-7: 0.001 | w_norm: 2.845


In [ ]:
def get_topk_gates(model, prune_sps=0.8):
    pruned_gates = dict()

    for name, param in model.named_parameters():
        if 'gsp_gate' in name:
            gate_vec = param.detach()
            k = math.floor(gate_vec.shape[0] * (1-prune_sps))
            val, ind = torch.topk(gate_vec, k=k)

            pruned_gates[name] = torch.zeros_like(gate_vec)
            pruned_gates[name][ind] = val

            pruned_gates[name] -= pruned_gates[name].min()
            pruned_gates[name] /= pruned_gates[name].max()
    # [x.shape for x in gates.values()]
    return pruned_gates

In [ ]:
def get_topk_act(model, prune_sps=0.8):
    pruned_gates = dict()
    pruned_gates_l = list()

    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            gate_vec = layer.reshaped_act.detach().flatten()
            k = math.floor(gate_vec.shape[0] * (1-prune_sps))
            val, ind = torch.topk(gate_vec, k=k)

            pruned_gates[name] = torch.zeros_like(gate_vec)
            pruned_gates[name][ind] = val

            pruned_gates[name] -= pruned_gates[name].min()
            pruned_gates[name] /= pruned_gates[name].max()
            pruned_gates_l.append(pruned_gates[name])
    # [x.shape for x in gates.values()]
    return pruned_gates, pruned_gates_l

In [ ]:
pruned_gates, pruned_gates_l = get_topk_act(model)
pruned_gates = get_topk_gates(model)
# [x.shape for x in pruned_gates.values()]

In [ ]:
def get_abs_sps(model):
    nonzero = total = 0
    # print(f"TYPE: {type(model)}")

    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            # print(name)
            tensor = layer.weight.detach().clone()
            # nz_count.append(torch.count_nonzero(tensor))
            nz_count = torch.count_nonzero(tensor).item()
            total_params = tensor.numel()
            nonzero += nz_count
            total += total_params
    
    # print(f"TOTAL: {total}")
    abs_sps = 100 * (total-nonzero) / total

    return abs_sps, total, (total-nonzero)

In [ ]:
# layer_d = dict()
def prune_filters(model):
    prod_l = list()
    act_mat_d = dict()
    prune_sps = 0.80

    for name, layer in model.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            # layer_d[name] = layer
            prod = torch.abs(layer.gsp_w @ layer.gsp_gate)
            # Normalize
            prod -= prod.min()
            prod /= prod.max()
            # print(f" gsp_w: {layer.gsp_w.shape} | gate: {layer.gsp_gate.shape} | prod: {prod.shape}")
            prod_l.append(prod)
            
            # Get TopK and create vector with topk values and zeros rest
            act_mat = torch.zeros_like(prod)
            k = math.floor(prod.shape[0] * (1 - prune_sps))
            vals, ind = torch.topk(prod, k=k)
            act_mat[ind] = vals

            if isinstance(layer, nn.Conv2d): 
                reshaped_act = act_mat.reshape(act_mat.shape[0],1,1,1)
                layer.weight.data = layer.weight.data * reshaped_act
            if isinstance(layer, nn.Linear):
                reshaped_act = act_mat.reshape(act_mat.shape[0],1)
                layer.weight.data = layer.weight.data * reshaped_act
            print(f"LayerShp: {layer.weight.shape} | prod: {prod.shape} | act_mat: {reshaped_act.shape}")
            # print(f"Layer Data Shape: {}")
            act_mat_d[name] = act_mat

In [ ]:
for name, layer in model.named_parameters():
    print(name)

In [ ]:
setup_pruning_exp(model, prune_sps = 0.9)

In [ ]:
for name, layer in model.named_parameters():
    print(name)

In [ ]:
prune_filters(model)
get_abs_sps(model)

In [ ]:
masks_d, masks_l = sps_tools.get_conv_linear_mask(model)

In [ ]:
apply_prune_mask(model, masks_l)

-----------
-----------

## Pass Minibatch to test GSP

In [ ]:
model.gsp_training_mode = True
model.start_gsp_epoch = -1

print(f"gsp start epoch: {model.start_gsp_epoch}")
print(f"epoch: {model.curr_epoch} | iter: {model.curr_iter} | gsp_int: {model.gsp_int}")

images, target = next(iter(trainloader))

images = images.cuda('cuda:0', non_blocking=True)
target = target.cuda('cuda:0', non_blocking=True)

optimizer.zero_grad()
output = model(images)
loss = criterion(output, target)

model.curr_iter += 1
get_abs_sps(model)

In [ ]:
# print(model.features[0].weight.grad)
# print(torch.norm(model.features[7].gsp_gate.grad))

In [ ]:
loss.backward()
optimizer.step()

get_abs_sps(model)

In [ ]:
print(torch.norm(model.features[7].gsp_gate.grad))
print(torch.norm(model.features[7].gsp_gate))

In [ ]:
# sps_tools.get_layerwise_sps(net)
# sps_tools.get_abs_sps(net)a

images, target = next(iter(trainloader))
images = images.cuda('cuda:0', non_blocking=True)
target = target.cuda('cuda:0', non_blocking=True)

# writer.add_graph(model, images)